In [ ]:
#-----------------------------------------------------------------------------------------------
# @Type: Quantum Software
# @Original Author: Dr. Jeffrey Chijioke-Uche, IBM Quantum Ambassador
# @Platform: Quantum Computing
# @License: Apache License 2.0 & Creative Commons Attribution-NonCommercial 4.0 International
#-----------------------------------------------------------------------------------------------

# After Qiskit Connector® pip install, Import Qiskit Connector®:
from qiskit_connector import QConnectorV2 as connector
from qiskit_connector import QPlanV2 as plan

# Initialize Qiskit Connector®:
current = plan()
backend = connector()

#-----------------------------------HOW TO USE QISKIT CONNECTOR--------------------------------------------------------------


#        This code sample is using the Qiskit Connector to run with a real quantum backend.
# ------------------------------------ QISKIT 2.x CODE SAMPLE ---------------------------------------------------------------
#############################################################################################################################
# Summary
# -------
# This script demonstrates Quantum Teleportation on an IBM real-device backend using Qiskit 2.x and the qiskit-connector v2.x.
# It builds a three-qubit circuit to teleport an arbitrary single-qubit state from qubit 0 to qubit 2 via entanglement and Bell
# measurements. The circuit is transpiled for the chosen backend (Open or Paid plan), submitted as a SamplerV2 job, and displays
# live spinner feedback while awaiting completion. Measurement outcomes from 1024 shots are aggregated and output as counts.
# Visualization adapts to the environment: textual histograms in a terminal or rich Matplotlib plots (histogram, pie chart,
# scatter plot) within Jupyter notebooks.
#
# Use Case
# --------
# This code provides a hands-on example of quantum information transfer—teleporting a qubit state—on IBM Quantum hardware.
# It’s ideal for researchers and educators exploring fundamental quantum communication protocols, demonstrating:
#  - How to configure and authenticate to IBM backends via qiskit-connector
#  - Constructing and transpiling conditional circuits on real QPUs
#  - Submitting and monitoring long-running SamplerV2 jobs with live feedback
#  - Aggregating and visualizing multi-shot measurement data in both CLI and notebook environments
#
# By running this teleportation protocol on actual hardware, users gain practical insight into gate fidelity, noise effects,
# and the end-to-end experience of quantum algorithm execution on cloud-accessible quantum processors.
#############################################################################################################################

from datetime import datetime
import time
import sys
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
from IPython.display import display, Markdown
from qiskit_ibm_runtime import SamplerV2 as Sampler, Session
import subprocess
import importlib.util

def in_jupyter():
    try:
        from IPython import get_ipython
        shell = get_ipython().__class__.__name__
        return shell in ('ZMQInteractiveShell',)  
    except Exception:
        return False

def automatic_pkg_install():
    core_pkg_import_array = ["IPython"]
    core_pkg_install_array = ["ipython"]
    if in_jupyter():
        try:
            for module in core_pkg_import_array:
                if importlib.import_module(module):
                    print(f"✅ {module} is already installed.")
        except ModuleNotFoundError:
            from IPython import get_ipython
            pip = get_ipython()
            for package in core_pkg_install_array:
                print(f"{package} not found. Attempting to install...")
                pip.run_line_magic('pip', f'install {package}')
            for module in core_pkg_import_array:
                if importlib.import_module(module):
                    print(f"✅ {module} now successfully installed.")
    else:
        try:
            for module in core_pkg_import_array:
                if importlib.import_module(module):
                   print(f"✅ {module} is already installed.")
        except ModuleNotFoundError:
            for package in core_pkg_install_array:
                print(f"{package} not found. Attempting to install...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            for module in core_pkg_import_array:
                if importlib.import_module(module):
                    print(f"✅ {module} now successfully installed.")

automatic_pkg_install()
def job_clear():
    sys.stdout.write('\r' + ' ' * 40 + '\r')
    sys.stdout.flush()


def job_inprogress():
    spinner = ['|', '/', '-', '\\']
    idx = 0
    print("Job in queue, waiting... ", end='', flush=True)
    while not job.done():
        sys.stdout.write(spinner[idx % len(spinner)])
        sys.stdout.flush()
        time.sleep(0.1)
        sys.stdout.write('\b') 
        idx += 1
    job_clear() 
    print("✅ Job successfully processed!           ")


def console_histogram(count_data, max_width=50):
    max_count = max(count_data.values())
    scale = max_width / max_count if max_count > 0 else 1

    print()
    for bitstring, count in sorted(count_data.items()):
        bar = '█' * int(count * scale)
        print(f"{bitstring:>5} | {bar} {count}")
      
shots = 1024

# Build Quantum Teleportation Circuit
qreg = QuantumRegister(3, 'q')
creg = ClassicalRegister(3, 'c')
qc = QuantumCircuit(qreg, creg)

# Step 1: Prepare the state to teleport
qc.h(qreg[0])
qc.barrier()

# Step 2: Create entanglement between qubit 1 and 2
qc.h(qreg[1])
qc.cx(qreg[1], qreg[2])
qc.barrier()

# Step 3: Bell measurement
qc.cx(qreg[0], qreg[1])
qc.h(qreg[0])
qc.barrier()

# Step 4: Measure qubits 0 and 1
qc.measure(qreg[0], creg[0])
qc.measure(qreg[1], creg[1])

# Step 5: Conditional corrections
qc.cx(qreg[1], qreg[2])
qc.cz(qreg[0], qreg[2])

# Step 6: Measure qubit 2
qc.measure(qreg[2], creg[2])

# Transpile for backend
transpiled_qc = transpile(qc, backend=backend, optimization_level=3)
print(f"✅ Circuit transpiled for {backend.name}")

# Draw circuit
render = "Quantum Teleportation Circuit"
def draw_circuit(circuit):
    if in_jupyter():
        display(Markdown(f"### {render}"))
        display(qc.draw(output="mpl"))
    else:
        print(f"\n{render}")
        print(qc.draw(output="text"))

# Run the circuit
if current == "Open Plan":
    print("⚡ Using Open Plan mode...")
    sampler = Sampler(mode=backend)
    job = Sampler(mode=backend).run([transpiled_qc], shots=shots)
    draw_circuit(qc)
    print("ok")
    job_inprogress()
else:
    print("⚡ Using Paid Plan mode with session...")
    with Session(backend=backend.name) as session:
        sampler = Sampler(mode=session)
        job = Sampler(mode=session).run([transpiled_qc], shots=shots)
        draw_circuit(qc)
        print("ok")
        job_inprogress()

# Get a real result:
results = job.result()
pub = results[0]
counts = pub.data.c.get_counts()
print("✅ Results retrieved:")
print(counts)

# Visualization
def display_results(counts, mode="histogram"):
    if in_jupyter():
        if mode == "pie":
            labels = list(counts.keys())
            sizes = list(counts.values())
            plt.figure(figsize=(6,6))
            plt.pie(sizes, labels=labels, autopct='%1.1f%%')
            plt.title("Quantum Teleportation - Pie Chart")
            plt.show()
        elif mode == "scatter":
            x = [int(k, 2) for k in counts.keys()]
            y = list(counts.values())
            plt.scatter(x, y)
            plt.xlabel("Outcome (decimal)")
            plt.ylabel("Counts")
            plt.title("Quantum Teleportation - Scatter Plot")
            plt.show()
    else:
        print("Results (text mode):", counts)


# Data Presentation
if in_jupyter():
    display(Markdown("\n📊 **Quantum Result Data Presentation**"))
    print("Quantum Teleportation - Histogram")
    display(plot_histogram(counts))
    display_results(counts, mode="pie")
    display_results(counts, mode="scatter")
else:
    print("\n📊 Histogram of Measurement Results")
    console_histogram(counts)
    print("\n📊 Pie Chart and Scattergraph require a GUI environment to display - Run this on Jupyter instead.")

# Footer
today = datetime.today().strftime("%B %d, %Y")
print(f"\nDr. Jeffrey Chijioke-Uche, IBM - Quantum Ambassador")
print(f"IBM Quantum Circuit - {today}")